<a href="https://colab.research.google.com/github/jcnborges/AutomaticHWCorrector/blob/main/desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
At the command line, only need to run once to install the package via pip:

$ pip install google-generativeai
"""

from pathlib import Path
import hashlib
import google.generativeai as genai

genai.configure(api_key="YOUR_API_KEY")
pdf_lista = "https://github.com/jcnborges/AutomaticHWCorrector/teste/Lista 1 - Revisão da linguagem Java.pdf"
zip_gabarito = "https://github.com/jcnborges/AutomaticHWCorrector/teste/gabarito.zip"
zip_respostas = "https://github.com/jcnborges/AutomaticHWCorrector/teste/respostas.zip"

# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

system_instruction = "Faça a avaliação das atividades como se fosse um professor de engenharia de software. Use o gabarito como modelo de resposta e o PDF para saber os enunciados de cada exercício. Para cada aluno, corrija os exercícios e atribua uma nota de 0 a 10, assim como média final. Depois que terminar a avaliação individual dos alunos, faça uma breve análise das dificuldades encontradas na turma como um todo."

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

prompt_parts = [
  f"--- INÍCIO DO PDF {pdf_lista} ---",
  f"--- INÍCIO DO ZIP {zip_gabarito} ---",
  f"--- INÍCIO DO ZIP {zip_respostas} ---",
]

response = model.generate_content(prompt_parts)
print(response.text)